# Análise de Portfólio de Ativos
Análise exploratória e indicadores de retorno, risco e performance de um portfólio de ativos financeiros.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Config(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Config(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

- Dados para Leitura

In [ ]:
Acoes = ["ITUB4.SA", "CPLE3.SA", "LREN3.SA", "EMBR3.SA", "PSSA3.SA", "GOAU4.SA"]
Periodo_Inicio = datetime.datetime(2016,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

- Dados do Portfolio

In [ ]:
PortfolioDistribuicao = [0.10, 0.20, 0.20, 0.10, 0.10, 0.30]
Portfolio = pd.DataFrame(data={"Ticker":Acoes, "Pesos":PortfolioDistribuicao}).set_index("Ticker")
Portfolio

In [ ]:
def LerAcoes(Acoes, Inicio, Termino):
    def LerAcao(Acao):
        return pdr.get_data_yahoo(Acao, start=Inicio, end=Termino)
    
    Dados = map(LerAcao, Acoes)
    
    return pd.concat(Dados, keys=Acoes, names=["Ticker", "Date"])

In [ ]:
DadosAcoes = LerAcoes(Acoes, Periodo_Inicio, Periodo_Termino)

In [ ]:
Dados = DadosAcoes.Close.reset_index().pivot(index="Date", columns="Ticker", values="Close")

In [ ]:
Dados

### Análise descritiva dos Preços
- Contagem de dias.
- Média dos preços do ativo.
- Desvio padrão dos preços.
- Preço Mínimo.
- Preço Máximo.
- Quartis dos Preços.

In [ ]:
Dados.describe()

In [ ]:
SeriePrecos = plg.Figure()
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["CPLE3.SA"], name="Copel", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["EMBR3.SA"], name="Embraer", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["GOAU4.SA"], name="Gerdau", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["ITUB4.SA"], name="Itau Holding", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["LREN3.SA"], name="Renner", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["PSSA3.SA"], name="Porto Seguro", mode="lines"))
Plot_Show(SeriePrecos, "Preços")

### Cálculo e análise descritiva dos Retornos
- Retorno Médio do Ativo (Média).
- Risco do Ativo (Desvio Padrão).
- Maior perda do período (Mínimo).
- Maior ganho no período (Máximo).
- Quartis do Risco.

In [ ]:
DadosRetorno = Dados.pct_change().fillna(0)

In [ ]:
DadosRetorno

In [ ]:
DadosRetorno.describe()

In [ ]:
SerieRetornos = plg.Figure()
SerieRetornos.add_trace(plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["CPLE3.SA"], name="Copel", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["EMBR3.SA"], name="Embraer", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["GOAU4.SA"], name="Gerdau", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["ITUB4.SA"], name="Itau Holding", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["LREN3.SA"], name="Renner", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["PSSA3.SA"], name="Porto Seguro", mode="lines"))
Plot_Show(SerieRetornos, "Retornos por Ativo")

### Retorno Acumulado por Ativo
Calcula dia-a-dia o retorno acumulado de cada ativo, sem ponderação da distribuição do portfólio, apenas usando o período.

In [ ]:
DadosRetornoAcumulado = (1+DadosRetorno).cumprod()-1

In [ ]:
DadosRetornoAcumulado

Retorno Acumulado de cada ativo

In [ ]:
DadosRetornoAcumulado[-1:]

In [ ]:
SerieRetornosAcum = plg.Figure()
SerieRetornosAcum.add_trace(plg.Scatter(x=DadosRetornoAcumulado.index, y=DadosRetornoAcumulado["CPLE3.SA"], name="Copel", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=DadosRetornoAcumulado.index, y=DadosRetornoAcumulado["EMBR3.SA"], name="Embraer", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=DadosRetornoAcumulado.index, y=DadosRetornoAcumulado["GOAU4.SA"], name="Gerdau", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=DadosRetornoAcumulado.index, y=DadosRetornoAcumulado["ITUB4.SA"], name="Itau Holding", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=DadosRetornoAcumulado.index, y=DadosRetornoAcumulado["LREN3.SA"], name="Renner", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=DadosRetornoAcumulado.index, y=DadosRetornoAcumulado["PSSA3.SA"], name="Porto Seguro", mode="lines"))
Plot_Show(SerieRetornosAcum, "Retornos Acumulados por Ativos")

### Indicadores de cada Ativo

- Retorno Médio (%)

In [ ]:
RetornoMedio = DadosRetorno.mean()
np.round(RetornoMedio * 100, 2)

- Risco (%)

In [ ]:
Risco = DadosRetorno.std()
np.round(Risco * 100, 2)

- Performance (Sharpe %)

In [ ]:
Sharpe = RetornoMedio / Risco / 100
np.round(Sharpe * 100, 2)

### Retorno Acumulado do Portfólio
Calcula dia-a-dia os retornos e retorno acumulado do portfólio, conforme distribuição nos ativos.

- Retorno dia-a-dia do portfólio

In [ ]:
PortfolioRetornos = Portfolio.transpose().dot(DadosRetorno.transpose()).transpose()

In [ ]:
DadosRetorno["Portfolio"] = PortfolioRetornos["Pesos"]

In [ ]:
DadosRetorno

In [ ]:
SerieRetornos = plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["Portfolio"], name="Portfólio", mode="lines")
Plot_Show(SerieRetornos, "Retorno do Portfólio")

In [ ]:
HistogramaRetornos = plg.Histogram(x=DadosRetorno["Portfolio"])
Plot_Show(HistogramaRetornos, "Retornos do Portfólio")

- Retorno acumulado do portfólio

In [ ]:
DadosRetorno["Acumulado"] = (1+DadosRetorno["Portfolio"]).cumprod()-1

In [ ]:
DadosRetorno

In [ ]:
DadosRetorno[-1:]

In [ ]:
SerieRetornos = plg.Scatter(x=DadosRetorno.index, y=DadosRetorno["Acumulado"], name="Portfólio", mode="lines")
Plot_Show(SerieRetornos, "Retorno Acumulado do Portfólio")

### Indicadores do Portfólio

- Retorno Médio

In [ ]:
RetornoMedio = DadosRetorno["Portfolio"].mean()
print("Retorno Médio: {0:.2%}".format(RetornoMedio))

- Risco

In [ ]:
Risco = DadosRetorno["Portfolio"].std()
print("Risco: {0:.2%}".format(Risco))

- Performance (Sharpe)

In [ ]:
Sharpe = RetornoMedio / Risco / 100
print("Performance Sharpe: {0:.2%}".format(Sharpe))

- VaR Histórico

In [ ]:
VarHistorico = np.quantile(DadosRetorno["Portfolio"], 0.05)
VarHistoricoPeriodos = np.sqrt([1,7,14,21]) * VarHistorico

InvestimentoPrevisto = 10000
PerdaMaxima = InvestimentoPrevisto * VarHistoricoPeriodos

print("VaR 1 dia: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[0], PerdaMaxima[0]))
print("VaR 7 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[1], PerdaMaxima[1]))
print("VaR 14 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[2], PerdaMaxima[2]))
print("VaR 21 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[3], PerdaMaxima[3]))

- VaR Paramétrico

In [ ]:
VarParam = RetornoMedio - norm.ppf(0.95) * Risco
VarParamPeriodos = np.sqrt([1,7,14,21]) * VarParam

InvestimentoPrevisto = 10000
PerdaMaxima = InvestimentoPrevisto * VarParamPeriodos

print("VaR 1 dia: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[0], PerdaMaxima[0]))
print("VaR 7 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[1], PerdaMaxima[1]))
print("VaR 14 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[2], PerdaMaxima[2]))
print("VaR 21 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[3], PerdaMaxima[3]))